In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split, cross_val_score
# from tpot.config.classifier_nn import classifier_config_nn
from sklearn.pipeline import make_pipeline
from tpot.config import classifier_config_dict_light
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_digits
import pandas as pd
import numpy as np
import os
import glob
from ipywidgets import IntProgress

In [ ]:
personal_config = classifier_config_dict_light

personal_config['sklearn.neural_network.MLPClassifier'] = {
    # MLPClassifier for neural networks
    # TODO: revisit/tweak: alpha, momentum, learning rate_init
    # separater paras based on activation
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'alpha': [1e-3, 1e-2, 1e-1, 1., 10., 100.],
    'learning_rate_init': [1e-3, 1e-2, 1e-1, 0.5, 0.75, 0.9],
    'momentum': [0.1, 0.5, 0.75, 0.9]
}


In [ ]:
path = ''
extension = 'csv'
os.chdir(path + "datasets/")
data_sets = [i.replace(".csv", "") for i in glob.glob('*.{}'.format(extension))]
print(data_sets[0:10])
os.chdir('..')

In [ ]:
for dat_name in data_sets:
    accuracy_ls = []
    tpot_data = pd.read_csv('datasets/' + dat_name + '.csv')
    Xdata = tpot_data.loc[:, tpot_data.columns != 'Class']
    Ydata = tpot_data['Class']
    X_train, X_test, y_train, y_test = train_test_split(Xdata, Ydata,
                                                        train_size=0.75, test_size=0.25)

    tpot = TPOTClassifier(generations=10, config_dict=personal_config,
                          population_size=10, verbosity=2,
                          template = 'Transformer-MLPClassifier')
    for i in range(10):
        tpot.fit(X_train, y_train)
        accuracy_ls.append([tpot._optimized_pipeline_score, tpot.score(X_test, y_test)])
        tpot.export('pipelines/' + dat_name + '_' + str(i) + '.py')

    accuracy_mat = pd.DataFrame(accuracy_ls, columns = ['Training CV Accuracy', 'Testing Accuracy'])
    accuracy_mat.to_csv("accuracies/" + dat_name + ".tsv", sep = "\t")




In [ ]:
# tpot_data = pd.read_csv(
#     'datasets/AAA_bench_mod_4_up_down_dt_svm_score_0.5538_60.txt', sep="\t")
# tpot_data = pd.read_csv(
#     'datasets/with_null_AAA_bench_mod_4_up_down_dt_gb_score_0.35173_75.csv')
# Xdata = tpot_data.loc[:, tpot_data.columns != 'Class']
# Xdata = Xdata.drop(['Fitness', 'Distribution'], axis=1)
# Ydata = tpot_data['Class']

# X_train, X_test, y_train, y_test = train_test_split(Xdata, Ydata,
#                                                     train_size=0.75, test_size=0.25)



In [ ]:
# tpot = TPOTClassifier(generations=10, config_dict=personal_config,
#                       population_size=10, verbosity=2,
#                       template='Selector-Transformer-MLPClassifier')
# tpot.fit(X_train, y_train)
# print(tpot.score(X_test, y_test))

# tpot.export('tpot_light_dt_gb_pipeline.py')